In [62]:
from re import search

from sympy.physics.quantum import gate

from aoc import *
from copy import deepcopy
from collections import defaultdict, Counter, deque
import re
from z3 import Ints, Solver, sat
from tqdm import tqdm
import itertools
from pprint import pprint

year = 2024
day = 24

download_input(year, day)

In [63]:
aoc, lines, G, R, C = read_input(day, test=False)

['stn AND ffg -> tnr', 'y43 XOR x43 -> vfw', 'x37 AND y37 -> gnn', 'x12 AND y12 -> knv', 'hqw AND jmq -> djd']
90 6


In [64]:
test = """x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02"""

test2 = """x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj"""

values, gates = aoc.split("\n\n")

val_dict = {}
for v_i in values.split("\n"):
    k, v = v_i.split(": ")
    val_dict[k] = int(v)


gate_list = []
for g in gates.split("\n"):
    x, op, y, _, out = [x.strip() for x in g.split(" ")]
    gate_list.append([x, op, y, out])
    if out.startswith("z"):
        val_dict[out] = None

# all_zs.sort()

while any(v is None for k, v in val_dict.items() if k.startswith("z")):
    for x, op, y, out in gate_list:

        if x not in val_dict or y not in val_dict:
            continue

        if op == "AND":
            val_dict[out] = val_dict[x] and val_dict[y]
        elif op == "OR":
            val_dict[out] = val_dict[x] or val_dict[y]
        elif op == "XOR":
            val_dict[out] = val_dict[x] ^ val_dict[y]
        else:
            assert False

# print(val_dict)

output = ""
for k in sorted(val_dict.keys(), reverse=True):
    if k.startswith("z"):
        output += str(val_dict[k])

print(int(output, 2))

55114892239566


In [65]:
# for each digit we need a full adder. Cin for z00 is empty.
# full adder is made up of a sum and a carry.

broken = {}

for gate in sorted(gate_list, key=lambda x: x[3]):
    if gate[3].startswith("z"):
        if gate[1] != "XOR" and gate[3] != "z45":
            broken[gate[3]] = None

# FINAL BIT MUST END WITH XOR. The following 3 gates are wrong.
# ['y08', 'AND', 'x08', 'z08']
# ['btj', 'AND', 'tmm', 'z16']
# ['jbc', 'OR', 'mnv', 'z32']

carries = {}
# These should be the carry bits.
for i in range(45):
    should_exist = [f"x{str(i).zfill(2)}", "XOR", f"y{str(i).zfill(2)}"]
    found = False
    for g in gate_list:
        if g[:3] == should_exist or g[:3][::-1] == should_exist:
            found = True
            carries[i] = g[3]
    if not found:
        print(f"XOR gate missing for {i}")

# every carry should be in an XOR
for c, carry in carries.items():
    if c == 0:
        continue
    found = False
    for gate in gate_list:
        if (gate[0] == carry or gate[2] == carry) and gate[1] == "XOR":
            found = True
    if not found:
        # print(carry)
        broken[carry] = None
        # print(f"Carry {carry} seems wrong.")

print(broken)

{'z08': None, 'z16': None, 'z32': None, 'dhm': None}


In [66]:
# So we have
# z08
# z16
# z32
# dhm
# as incorrect outputs.
# The z bits should be detectable.

ok = set()
for b in broken:
    # Incorrect XORs
    try:
        digit = int(b[1:])

        should_be = [f"x{str(digit).zfill(2)}", "XOR", f"y{str(digit).zfill(2)}"]

        for g in gate_list:
            if g[:3] == should_be or g[:3][::-1] == should_be:
                for gg in gate_list:
                    if (gg[0] == g[3] or gg[2] == g[3]) and gg[1] == "XOR":
                        broken[b] = gg[3]
    # Incorrect carries
    except:
        for g in gate_list:
            if g[3] == b:
                should_be = [g[0], "AND", g[2]]
                for gg in gate_list:
                    if gg[:3] == should_be or gg[:3][::-1] == should_be:
                        broken[b] = gg[3]

print(",".join(sorted(list(broken.keys()) + list(broken.values()))))

cdj,dhm,gfm,mrb,qjd,z08,z16,z32


In [70]:
test = """x00: 0
x01: 1
x02: 0
x03: 1
x04: 0
x05: 1
y00: 0
y01: 0
y02: 1
y03: 1
y04: 0
y05: 1

x00 AND y00 -> z05
x01 AND y01 -> z02
x02 AND y02 -> z01
x03 AND y03 -> z03
x04 AND y04 -> z04
x05 AND y05 -> z00"""

values, gates = aoc.split("\n\n")


def reset_val_dict_and_gate():
    val_dict = {}
    for v_i in values.split("\n"):
        k, v = v_i.split(": ")
        val_dict[k] = int(v)

    gate_list = []
    for g in gates.split("\n"):
        x, op, y, _, out = [x.strip() for x in g.split(" ")]
        gate_list.append([x, op, y, out])
        if out.startswith("z"):
            val_dict[out] = None

    return val_dict, gate_list


val_dict, gate_list = reset_val_dict_and_gate()

starting_x_b = "".join(
    [str(v) for k, v in sorted(val_dict.items(), reverse=True) if k.startswith("x")]
)
starting_x = int(starting_x_b, 2)
starting_y_b = "".join(
    [str(v) for k, v in sorted(val_dict.items(), reverse=True) if k.startswith("y")]
)
starting_y = int(starting_y_b, 2)

print(starting_x_b, starting_x)
print(starting_y_b, starting_y)

desired_z = starting_x + starting_y
desired_z_b = np.base_repr(desired_z, base=2)
print(desired_z_b, desired_z)


search = None

change_indices = set()
for a, b in broken.items():
    # print(a,b)
    indices = set()
    for g, gate in enumerate(gate_list):
        if gate[3] == a or gate[3] == b:
            indices.add(g)
        if len(indices) == 2:
            change_indices.add(frozenset(indices))

for i, j in change_indices:
    temp = gate_list[i][3]
    gate_list[i][3] = gate_list[j][3]
    gate_list[j][3] = temp

while any(v is None for k, v in val_dict.items() if k.startswith("z")):

    updates = 0
    for x, op, y, out in gate_list:

        if x not in val_dict or y not in val_dict:
            continue

        # if out not in val_dict:
        if op == "AND":
            val_dict[out] = val_dict[x] & val_dict[y]
            updates += 1
        elif op == "OR":
            val_dict[out] = val_dict[x] | val_dict[y]
            updates += 1
        elif op == "XOR":
            val_dict[out] = val_dict[x] ^ val_dict[y]
            updates += 1
        else:
            assert False

    if updates == 0:
        print("Breaking")
        break

# print(val_dict)

output = ""
ans = [x for x, a in val_dict.items() if x.startswith("z") and a is not None]
print(output)
if len(ans) == 46:
    for k in sorted(ans, reverse=True):
        if k.startswith("z") and val_dict[k] is not None:
            output += str(val_dict[k])

    search = int(output, 2)
    if search == desired_z:
        print("success", broken)
        print(",".join(sorted(list(broken.keys()) + list(broken.values()))))


#
# print(int(output,2))

110101101010010100111101100100111111010101111 29500648881839
101110000101011000111111000000110010000011111 25335070483487
1100011101111101101111100100101110001011001110 54835719365326

success {'z08': 'cdj', 'z16': 'mrb', 'z32': 'gfm', 'dhm': 'qjd'}
cdj,dhm,gfm,mrb,qjd,z08,z16,z32
